In [21]:
%matplotlib notebook

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import scipy.stats as sts


# Import API key
from api_keys import g_key
from api_keys import api_key
from api_keys import weather_api_key
#gmaps.configure(api_keys= g_key)


In [19]:
city_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#data =  city_url + "lompoc" + "&APPID=" + api_key
#response = requests.get(city_url)
#data = response.json()
#pprint(data)

#import openweathermapy as gmaps


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
#Loading in WeatherPY CSV

input_file="cities.csv"


csv=pd.read_csv(input_file)

csv.head()


,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,81.34,71,92,7.78,MV,1627858049
1,1,Yellowknife,62.4560,-114.3525,87.33,28,20,3.44,CA,1627858059
2,2,Saint-Joseph,-21.3667,55.6167,64.78,75,37,8.30,RE,1627858292
3,3,Cidreira,-30.1811,-50.2056,56.86,93,90,8.03,BR,1627858086
4,4,Talara,-4.5772,-81.2719,65.91,84,1,21.12,PE,1627858137


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Creating necesarry variables for Location and Humidity
locations=csv[["Lat", "Lng"]]
Humidity= csv["Humidity"]

#Define the heatmap parameters
fig=gmaps.figure()

#Create heat layer
heat_layer=gmaps.heatmap_layer(locations, weights=Humidity, dissipating= False, max_intensity=250,
                              point_radius=3)

#Add Layer
fig.add_layer(heat_layer)

#Display the map
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#Personal preference
    #80> but not higher 90 
    #50< Humidity
    #Lat > 20

temp_df=csv[(csv["Max Temp"]>=80)&(csv["Max Temp"]<90)&(csv["Humidity"]<50)& (csv["Lat"]>20)]
temp_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#create df of hotel name, along with city, count, geocod(lat,long)
hotel_df=temp_df[["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Names"]= ""
hotel_df

In [ ]:
#Define variables
radius=8000
gtype="lodging"

#Making query calls for top Hotel

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={"radius": radius, "types": gtype,"key": g_key}

In [ ]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_df=hotel_df[hotel_df["Hotel Names"]!= ""]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig